In [ ]:
"""
Author: Liam Bogucki
Email: lboguck@uwo.ca
First Written: Friday, July 5, 2024
Last Modified: Monday, August 18, 2025
Program Purpose: To convert the temperature dataset into one that can be used to partion the tropical forest regions as done by Ahlstrohm.
"""

In [ ]:
#Importing the appropriate libraries
import matplotlib.pyplot as plt
import numpy as np
import netCDF4 as nc
import xarray as xr

# Changing over the font
plt.rcParams['font.family'] = 'Times New Roman'

In [ ]:
#This block of code handles the initial loading of the temperature data and the specific selection of the 2 metre temperature

#Loading the temperature data
temperature_data = nc.Dataset("temp.nc", "r")
#Looking at the dataset
print(type(temperature_data))
print(temperature_data.variables) #The values that are needed are the t2m (2 metre temperature)

#Getting the 2m temperature to handle
m2_temperature = temperature_data.variables['t2m'][:]
#Looking at what the temperature looks like
print(np.shape(m2_temperature)) #The dimensions are (1013, 2, 721, 1440) : (months, ERA5||ERA5T, lat, long)

In [ ]:
#This block handles the processing of the temperature data down to 3 dimensions (months, lat, long)

#Verifying array
print(type(m2_temperature)) #Masked numpy array
#Merging temperatures down to one dimension for the temperatures
dimensions = ['months', 'dataset', 'lat', 'long'] #Labels for dimensions based on retrieved values above
m2_temperature = xr.DataArray(m2_temperature, dims=dimensions) #Converting to xarray to merge
m2_temperature_new = m2_temperature.reduce(np.nansum, dim='dataset') #Reducing down to 3d array (merge of temp data)

#Verifying the new array
print(type(m2_temperature_new))
print(np.shape(m2_temperature_new)) #Now has dims(1013, 721, 1440)

In [ ]:
#This block handles the truncation of the array to match the desired 360x720 required for our analysis

#Converting the xarray back to a numpy array for trimming of the bottom par of the array
m2_3d_temperature = m2_temperature_new.to_numpy()
print(type(m2_3d_temperature)) #verifying is numpy
print(np.shape(m2_3d_temperature)) #verifying the dims (1013, 721, 1440)

#Since 720 instead of 721 is needed, slicing the bottom of the array off
m2_3d_temperature = m2_3d_temperature[:, :-1, :]
print(np.shape(m2_3d_temperature)) #verifying the cutoff

#Converting the array back to an xarray to average the data to the correct dimensions
dimensions = ['months', 'lat', 'long'] #Labels for dimensions based on retrieved values above
m2_3d_temperature = xr.DataArray(m2_3d_temperature, dims=dimensions) #Converting to xarray
m2_3d_temperature = m2_3d_temperature.coarsen(lat=2, long=2).mean()

#Converting back to numpy array and verifying the appearence
m2_3d_temperature = m2_3d_temperature.to_numpy()
print(type(m2_3d_temperature)) #verifying is numpy
print(np.shape(m2_3d_temperature)) #verifying the dims (1013, 360, 720)
plt.imshow(m2_temperature_new[10][:][:])#looking at what the temperature looks like for a random month (notice that will need to role below)

In [ ]:
# This block handles the rolling of the array such that the layout of the map aligns with the MODIS data (and save as the new temp dataset)

#Rolling the array to have it align with the MODIS data that has already been processed
m2_3d_temperature = np.roll(m2_3d_temperature, shift=-359, axis=2) #rolling the longitude to align with MODIS layout (Plate Carree)

#Verifying the MODIS and temp overlap is correct
data_modis = np.load("Modis_0.5Deg_Type3_Mode.npy")
print(np.shape(data_modis))
temp = m2_3d_temperature[10,:,:] # Randomly using the 10th month to compare to
temp[data_modis ==  0] = 999 # Removing the water area to see alignment
print(np.shape(temp))
plt.imshow(temp) #Can see that the map looks as it should
plt.show()

#Removing the water from all of the months and saving as the temp array
data_modis_3d = np.tile(data_modis, (1013, 1, 1))
m2_3d_temperature[data_modis_3d==0] = np.nan #Setting to nan where water based on the modis data == 0
np.save('temp_data_monthly.npy', m2_3d_temperature) #saving for further use in our analysis

#Final verification of the layout that we will use
plt.imshow(data_modis_3d[0, :, :])